In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install opencv-python-headless
!pip install mtcnn

In [ ]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
import json

detector = MTCNN()

In [ ]:
cv2.setUseOptimized(True)
cv2.setNumThreads(4)
cv2.ocl.setUseOpenCL(True)

In [ ]:
TARGET_SIZE = (300, 300)

def test_preprocess_video(video_path):
    frames = []
    frame_count = 0


    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()

        if not ret:
            break


        results = detector.detect_faces(frame)

        if results:
            x, y, w, h = results[0]['box']
            face = frame[y:y+h, x:x+w]


            if len(face.shape) == 2:
                face = cv2.cvtColor(face, cv2.COLOR_GRAY2BGR)


            face = cv2.resize(face, TARGET_SIZE)

            frames.append(face)

        frame_count += 1

    cap.release()
    return frames


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Flatten, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

DATA_FOLDER = "gdrive/My Drive/ColabDrive/deepfake-detection-challenge/train_colab_videos/"
MODEL_FILE = "my_model_one.h5"

test_model = load_model(os.path.join(DATA_FOLDER, MODEL_FILE))

In [ ]:
test_video_path = 'gdrive/My Drive/ColabDrive/deepfake-detection-challenge/train_sample_videos/aagfhgtpmv.mp4'

test_frames = test_preprocess_video(test_video_path)
test_data = np.array(test_frames)

In [ ]:
frame_predictions_list = test_model.predict(test_data)

In [ ]:
real_frames = 0
fake_frames = 0

for frame_predictions in frame_predictions_list:
    frame_real = 0
    frame_fake = 0

    for prediction in frame_predictions:
        if prediction[0] > 0.5:
            frame_real += 1
        else:
            frame_fake += 1

    if frame_real > frame_fake:
        real_frames += 1
    else:
        fake_frames += 1

if real_frames > fake_frames:
    video_category = "REAL"
else:
    video_category = "FAKE"

print("Video Category:", video_category)

